In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default='notebook'

#
# Used to implement the multi-dimensional counter we need in the performance class
from collections import defaultdict
from functools import partial
from itertools import repeat
def nested_defaultdict(default_factory, depth=1):
    result = partial(defaultdict, default_factory)
    for _ in repeat(None, depth - 1):
        result = partial(defaultdict, result)
    return result()

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

2.6.0
2.6.0


In [2]:
file1 = 'Player_' + str(1) + '.csv'
df1 = pd.read_csv(file1, header=None, skiprows=2)
headers = df1.iloc[0]
df1 = pd.DataFrame(df1.values[1:], columns=headers)
player_str = "Player " + str(1) + " played:"
df_data = pd.DataFrame(columns=["Player_Choice", "CPU_Choice"])
df_data["Player_Choice"] = df1[player_str].replace(["R", "P", "S"],[0,1,2])
df_data["CPU_Choice"] = df1["Multi AI played:"].replace(["R", "P", "S"],[0,1,2])

In [3]:
df_sub = pd.DataFrame(columns=["Player_Choice", "CPU_Choice"])
for i in range(2,42):
    file = 'Player_' + str(i) + '.csv'
    df = pd.read_csv(file, header=None, skiprows=1)
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)
    player_str = "Player " + str(i) + " played:"
    df_sub["Player_Choice"] = df[player_str].replace(["R", "P", "S"],[0,1,2])
    df_sub["CPU_Choice"] = df["Multi AI played:"].replace(["R", "P", "S"],[0,1,2])
    
    frames = [df_data, df_sub]
    df_data = pd.concat(frames, ignore_index=True)
    
df_data.head(len(df_data))

df_data.to_csv('large_player_data.csv') 

In [4]:
import random

train_len = int(0.75 * len(df_data['Player_Choice']))
test_len = int(len(df_data['Player_Choice']) - train_len)
print(train_len, test_len)

foo = df_data['Player_Choice'].tolist()
bar = df_data['CPU_Choice'].tolist()

train_data = []
train_labels = []

for i in range(train_len):
    x = random.randint(0,(train_len-8))
    y_1 = []
    y_2 = []
    for j in range(x, x+8):
        if j < x+7:
            y_1.append(foo[j])
        else:
            y_2.append(bar[j])
    train_data.append(y_1)
    train_labels.append(y_2)

    
test_data = []
test_labels= []

for i in range(train_len, train_len+test_len):
    x = random.randint(train_len, train_len+test_len-8)
    y_1 = []
    y_2 = []
    for j in range(x, x+8):
        if j < x+7:
            y_1.append(foo[j])
        else:
            y_2.append(bar[j])
    test_data.append(y_1)
    test_labels.append(y_2)
    
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

9225 3075


In [5]:
print("Train info",train_data.shape, train_labels.shape)
print("Test info",test_data.shape, test_labels.shape)

Train info (9225, 7) (9225, 1)
Test info (3075, 7) (3075, 1)


In [6]:
#train_data = train_data.reshape((train_data.shape[0],28,28,1))
train_data = train_data.astype('float32')/2
#
#test_data = test_data.reshape((test_data.shape[0],28,28,1))
test_data = test_data.astype('float32')/2
print("test_labels",test_labels[:5])

train_labels_cat = keras.utils.to_categorical(train_labels)
test_labels_cat = keras.utils.to_categorical(test_labels)

test_labels [[1]
 [2]
 [2]
 [0]
 [0]]


In [7]:
print(train_labels_cat.shape)

(9225, 3)


In [8]:
#print(test_labels)

In [16]:
from keras import models
from keras import layers

# Define the model
fcn_network = models.Sequential()
#
fcn_network.add(layers.Flatten(input_shape=[7]))
#
# Hidden
fcn_network.add(layers.Dense(100,activation='tanh'))
fcn_network.add(layers.Dense(400,activation='tanh'))
fcn_network.add(layers.Dense(100,activation='tanh'))
#
# Output
fcn_network.add(layers.Dense(3,activation='softmax'))
#
# Compile
fcn_network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# 
# Fit/save/print summary
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
fcn_results = fcn_network.fit(train_data,train_labels_cat,
                              epochs=100,
                              batch_size=128,
                              callbacks=callbacks, # Early stopping
                              validation_data=(test_data,test_labels_cat))
fcn_network.save('Large_model_fcn.h5')

Epoch 1/100
73/73 [==============================] - 1s 17ms/step - loss: 1.1056 - accuracy: 0.3639 - val_loss: 1.1004 - val_accuracy: 0.3824
Epoch 2/100
73/73 [==============================] - 1s 16ms/step - loss: 1.0987 - accuracy: 0.3720 - val_loss: 1.1007 - val_accuracy: 0.3584
Epoch 3/100
73/73 [==============================] - 1s 17ms/step - loss: 1.0942 - accuracy: 0.3716 - val_loss: 1.1060 - val_accuracy: 0.3502
Epoch 4/100
73/73 [==============================] - 1s 16ms/step - loss: 1.0931 - accuracy: 0.3708 - val_loss: 1.0975 - val_accuracy: 0.3636
Epoch 5/100
73/73 [==============================] - 1s 17ms/step - loss: 1.0933 - accuracy: 0.3688 - val_loss: 1.1011 - val_accuracy: 0.3548
Epoch 6/100
73/73 [==============================] - 1s 17ms/step - loss: 1.0911 - accuracy: 0.3775 - val_loss: 1.0998 - val_accuracy: 0.3678
Epoch 7/100
73/73 [==============================] - 1s 16ms/step - loss: 1.0903 - accuracy: 0.3783 - val_loss: 1.0975 - val_accuracy: 0.3454
Epoch 

In [10]:
from keras.models import load_model
test_list = [[1,2,0,2,0,1,1]]
test_list = np.array(test_list)

network_name = 'Large_model_fcn.h5'
fcn_network = load_model(network_name)

print(test_list.shape)

(1, 7)


In [11]:
predictions = fcn_network.predict(test_list)

In [12]:
print(predictions)

[[0.48666367 0.27419823 0.23913807]]


In [13]:
print(np.argmax(predictions))

0
